# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [85]:
#импорт библиотек
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [86]:
df = pd.read_csv('C:\\Users\\User\\Downloads\\Yandex_projects\\data.csv') # чтение файла

In [87]:
df.head(10) # вывод первых 10 строк табоицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [88]:
df.info() # общие сведения о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [89]:
df.describe() # данные о среднем арифметическом, стандартном отклонении, минимальном занчении, первом, втором и третьем процентилях
# и максимальном значении в каждом из столбцов 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В датасете имеются странные значения (например, отрицательные значения в столбце с общим трудовым стажем в днях), которые при необходимости надо обработать, также как минимум в столбце education есть неявные повторы (например, "среднее", "Среднее" и "СРЕДНЕЕ" по смыслу одинкаовы, но будут воприниматься pandas как различные значения). По выводы df.info() можно сказать, что в таблице имеются пропуски, потому что не во всех столбцах совпадает количество непустых значений.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [90]:
print(df.isna().sum()) # поиск количества пропусков в каждой колонке
print()
# пропуски располагаются в столбцах days_employed и total_income
df['days_employed'] = df['days_employed'].fillna(value=0) #заполнение пропусков в столбце days_employed нулями
median_income = df.groupby('income_type')['total_income'].median() # медианный доход по какой группе типа занятости
# заполнение пропусков в столбце total_income медианным значением в соответствии с типом занятости
for inc_type in median_income.index:
    df.loc[df['income_type'] == inc_type, 'total_income'] = df.loc[df['income_type'] == inc_type, 'total_income'].fillna(median_income[inc_type])
print()
print(df.isna().sum()) # проверка заполнения пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


In [91]:
df['dob_years'].unique() # уникальные значения столбца dob_years
# обнаружены заемщики со значением возраста равным нулю, что означает пропущенные значения возраста
# обнако этот столбец в дальшем не понадобится, поэтому оставим столбец без изменений

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [92]:
df['gender'].unique() # уникальные значения стобца gender 
# обнаружено странное значение пола XNA (пол неизвестен)
# этот столбец также не понадобится, поэтому оставим значение с неизвестным полом как есть

array(['F', 'M', 'XNA'], dtype=object)

In [93]:
df['children'].unique() # уникальные значения столбца children
# обнаружены подозрительные значения -1 и 20 

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [94]:
# замена значений
df.loc[df['children'] == -1, 'children'] = 1 # замена количества детей равного -1 на 1
df.loc[df['children'] == 20, 'children'] = 2 # замена количества детей равного 20 на 2
df['children'].unique() # проверка замены значений

array([1, 0, 3, 2, 4, 5], dtype=int64)

**Вывод**

Пропуски распологаются в столбцах days_employed и total_income. Количество пропусков в столбце days_employed равно количетсву пропусков в столбце total_income: если в строке встречатся пропуск в столбце days_employed, то встретится пропуск и в столбце total_income. Вероятно, это связано с тем, что банк может выдавать кредит без справок о доходах.
Пропущенные значения в столбце days_employed были заполнены нулями, так как значения из этого столбца в дальнейшем не понадобятся. Пропущенные значения в столбце total_income были заменены на медианные значения доходов для каждой категории занятости. В столбце dob_years есть ячейки со значением 0. Этот столбец в дальнейшем не понабодится, поэтому он был оставлен без изменений.

### Замена типа данных

In [95]:
# замена вещественного типа данных на целочисленный в колонках days_employed и total_income с помощью метода astype()
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info() # проверка замены типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод**

Для замены вещественного типа данных на целочисленный использовался метод astype(). Этот метод был выбран, так как метод astype() работает со всеми форматами данными.

### Обработка дубликатов

In [96]:
# нахождение уникальных значение в столбцах с типом данных object
df['education'].value_counts() #уникальные значения столбца education
# в столбце education пристуствуют значения одинаковые по смыслу, но записанные в разных регистрах


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [97]:
df['family_status'].value_counts() #уникальные значения столбца family_status
# аналогичная ситуация в столбце family_status

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [98]:
df['income_type'].value_counts() #уникальные значения столбца income_type
# все значения в столбце income_type в одном регистре

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [99]:
df['purpose'].value_counts() #уникальные значения столбца purpose
# в столбце value_counts все значения так же в одном регистре

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [100]:
# так как в столбцах education и family_status есть одинаковые значение в разном регистре, то необходимо перевести эти значения в нижний регистр с помощью метода str.lower()
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
#проверка замены регистра

print('Уникальные значения столбца education без учета регистра')
print(df['education'].value_counts())
print()
print('Уникальные значения столбца family_status без учета регистра')
df['family_status'].value_counts()


Уникальные значения столбца education без учета регистра
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Уникальные значения столбца family_status без учета регистра


женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [101]:
print('Количество дубликатов до их удаления:', df.duplicated().sum()) # подсчет дубликатов в обновленной таблице
df = df.drop_duplicates().reset_index(drop=True) # удаление дубликатов с помощью метода drop_duplicates()
print('Проверка удаления дубликатов:', df.duplicated().sum()) # проверка успешного удаления дубликатов

Количество дубликатов до их удаления: 71
Проверка удаления дубликатов: 0


In [102]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

**Вывод**

Чтобы правильно удалить все дубликаты, сначала необходимо привести к одному регистру ячейки с одинаковыми значениями. В противном случае такие ячейки будут восприниматься pandas как разные значения, что приведет к неполному удаления дубликатов. Уникальные значения в каждом столбце с типом данных object выводятся с помощью метода value_counts(). Из пяти таких столбцов было обнаружено два столбца (education и family_status), где присутствуют буквы в верхнем регистре (например, "среднее", "Среднее" и "СРЕДНЕЕ" стали "среднее"). Для приведения всех текстовых ячеек в этих столбцах в нижний регистр был использован метод str.lower(). После того, как все текстовые ячейки были переведы в нижний регистр, дубликаты были удалены с помощью метода drop_duplicates(). Также совместно с drop_duplicates() был вызван метод reset_index с аргументом drop=True, чтобы не создавать столбец со старыми индексами.

Дубликаты могли появиться из-за того, что некоторые люди могли оставлять заявку на кредит в банке больше одного раза.


### Лемматизация

In [103]:
m = Mystem()

text_lemmas = '' # текстовая строка, куда будут записываться значения ячеек из столбца purpose
for i in range(len(df)):
    text_lemmas += df.loc[i, 'purpose'] + ' '

In [104]:
lemmas = m.lemmatize(text_lemmas) # лемматизация строки text_lemmas
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, ' \n': 1})


In [105]:
purpose_lem = []
for i in df['purpose']:
    result = ''.join(m.lemmatize(i)).strip()
    purpose_lem.append(result)
df['purpose_lem'] = purpose_lem
df

KeyboardInterrupt: 

In [82]:
lemmas_general = ['свадьба', 'автомобиль', 'образование', 'недвижимость'] # более общие цели кредита

# создание столбца purpose_lemma, где будут храниться лемматизированные значения ячеек из столбца purpose
df['purpose_lemma'] = df['purpose'].apply(m.lemmatize)
print(df)

KeyboardInterrupt: 

In [61]:
# импорт библиотеки pymystem3 
#from pymystem3 import Mystem
# m = Mystem()

# text_lemmas = '' # текстовая строка, куда будут записываться значения ячеек из столбца purpose
# for i in range(len(df)):
#     text_lemmas += df.loc[i, 'purpose'] + ' '
# lemmas = m.lemmatize(text_lemmas) # лемматизация строки text_lemmas
# # подсчет количества раз, которое встречается каждая лемма
# #from collections import Counter
# #print(Counter(lemmas))

# lemmas_general = ['свадьба', 'автомобиль', 'образование', 'недвижимость'] # более общие цели кредита

# # создание столбца purpose_lemma, где будут храниться лемматизированные значения ячеек из столбца purpose
# df['purpose_lemma'] = df['purpose'].apply(m.lemmatize)
# #print(df)

# def general_purpose(purpose_lemma): #функция general_purpose находит пересечение списка lemmas_general и списка, находящегося в каждой ячейке столбца purpose_lemma
#     for i in lemmas_general:
#         intersection = ''
#         if i in purpose_lemma:
#             intersection = i
#             return intersection
# df['purpose_general'] = df['purpose_lemma'].apply(general_purpose) # создание столбца purpose_general, в котором будут храниться общие название для целей кредита
# df.loc[df['purpose_general'] == 'None', 'purpose_general'] = 'недвижимость'
# print(df)

KeyboardInterrupt: 

**Вывод**

Чтобы провести лемматизацию целей кредита, сначала все значения столбца purpose были добавлены в строку text_lemmas, так как метод lemmatize() принимает аргументы в формате строки. Затем аргумент text_lemmas был лемматизирован при момощи метода lemmatize(). С помощью Counter была подсчитана частота встречаемости лемматизированных слов. Из результата работы Counter (список lemmas_general) можно выделить пять общих групп для всех целей кредитов: жилье, свадьба, автомобиль, образование, недвижимость. 
Затем был создан столбец purpose_lemma, в который были записаны списки из лемматизированных целей кредита. Далее необходимо сопоставить значения purpose_lemma значениям из списка lemmas_general. 
Функция general_purpose ищет пересение ячеек purpose_lemma со значениями из lemmas_general. Результат работы функции записывается в столбец purpose_general.

In [20]:
df['purpose_general'] = df['purpose_general'].fillna('недвижимость')
df

KeyError: 'purpose_general'

### Категоризация данных

In [ ]:
# создание словаря уровней образования: ключи - цифровое обозначение уровня образования, значения - словесное обозначение уровня образования
education_dict = df[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print('Словарь для уровня образования')
education_dict

In [22]:
# создание словаря семейных положений: ключи - цифровое обозначение семейного положения, значения - словесное обозначение семейного положения
family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print('Словарь для семейного положения')
family_status_dict

Словарь для семейного положения


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [22]:
print('Словарь категорий доходов')
df['income_category']= pd.cut(df['total_income'], bins = [0, 100000, 200000, 500000], labels=['low', 'medium', 'high']) # разделение клиентов на три группы доход: 'low' - до 100 тысяч, 'medium' - от 100 до 200 тысяч, 'high' - больше 200 тысяч
income_dict = df[['income_category', 'total_income']] # формирование словаря категорий доходов
income_dict = income_dict.drop_duplicates().reset_index(drop=True)
income_dict

Словарь категорий доходов


,income_category,total_income
0,high,253875
1,medium,112080
2,medium,145885
3,high,267628
4,medium,158616
...,...,...
18603,high,224791
18604,medium,155999
18605,low,89672
18606,high,244093


***Я переделала словарь для категорий доходов: с помощью метода cut разбила доходы на категории 'low', 'medium' и'high'***

**Вывод**

Было выделено три словаря: словарь уровня образования, словарь семейного положения и словарь медиан по категориям доходов. Словарь уровня образования и словарь семейного положения были сгруппированы, так как в исходном датафрейме есть и численное, и словесное описание уровня образования и семейного положения соответственно. В словаре медиан по категориям доходов в столбце income_type были выделены уникальные значения типов занятости, и по каждому типу занятости были посчитаны медианные значения зарплаты методом median().

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
df_children_debt = df.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count') # формирование сводной таблицы по столбцам children и debt
#print(df_children_debt) #проверка получившейся сводной таблицы
df_children_debt[1] = df_children_debt[1].fillna(value=0) # замена NaN в столбце 1 на 0
def percent_debt(row): # функция, которая считает процент должников от общего количества заемщиков по каждой строке
    no_debt = row[0]
    yes_debt = row[1]
    percent_int = 100*yes_debt/(yes_debt+no_debt)
    percent = str(round(percent_int, 2)) + "%"
    return percent
df_children_debt['debt_percent'] = df_children_debt.apply(percent_debt, axis=1) # добавление столбца с процентами должников в таблицу df_children_debt
print(df_children_debt)

debt            0       1 debt_percent
children                              
0         13028.0  1063.0        7.54%
1          4410.0   445.0        9.17%
2          1926.0   202.0        9.49%
3           303.0    27.0        8.18%
4            37.0     4.0        9.76%
5             9.0     0.0         0.0%


In [24]:
df_example = df.groupby('children')['debt'].agg(['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.reset_index().style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,children,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
0,0,14091,1063,7.54%,92.46%
1,1,4855,445,9.17%,90.83%
2,2,2128,202,9.49%,90.51%
3,3,330,27,8.18%,91.82%
4,4,41,4,9.76%,90.24%
5,5,9,0,0.00%,100.00%


In [25]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_general
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


In [37]:
df_children_income = df.pivot_table(index=['income_type'], columns='children', values='dob_years', aggfunc='count')
df_children_income = df_children_income.fillna(value='-')
df_children_income['median_income'] = df.groupby('income_type')['total_income'].median()
df_children_income

children,0,1,2,3,4,5,median_income
income_type,,,,,,,
безработный,1,1,-,-,-,-,131339
в декрете,-,-,1,-,-,-,53829
госслужащий,866,358,189,36,7,1,150447
компаньон,3137,1307,551,79,2,2,172357
пенсионер,3510,283,29,6,1,-,118514
предприниматель,2,-,-,-,-,-,499163
сотрудник,6574,2906,1358,209,31,6,142594
студент,1,-,-,-,-,-,98201


**Вывод**

Количество должников в процентном соотношении увеличивается с увеличением количества детей, при том, что люди с одним ребенок, как и люди с пятью детьми нередко находятся в одной категории доходов (в категории 'medium'). Возможно, это связано с тем, что с увеличением количества детей возрастают расходы, и у заемщиков становится меньше средств на возраст долга.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
df_family_debt = df.pivot_table(index=['family_status'], columns='debt', values='dob_years', aggfunc='count') # создание сводной таблицы по семейному положению и количеству должников в зависимости от семейного положения
df_family_debt['debt_percent'] = df_family_debt.apply(percent_debt, axis=1) # создание столбца с процентом должников от общего числа заемщиков по каждой категории семейного положения 
print(df_family_debt)


debt                       0    1 debt_percent
family_status                                 
в разводе               1110   85        7.11%
вдовец / вдова           896   63        6.57%
гражданский брак        3763  388        9.35%
женат / замужем        11408  931        7.55%
не женат / не замужем   2536  274        9.75%


In [28]:
df_family_and_debt = df.groupby('family_status')['debt'].agg(['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_family_and_debt.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_family_and_debt.reset_index().style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,family_status,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
0,в разводе,1195,85,7.11%,92.89%
1,вдовец / вдова,959,63,6.57%,93.43%
2,гражданский брак,4151,388,9.35%,90.65%
3,женат / замужем,12339,931,7.55%,92.45%
4,не женат / не замужем,2810,274,9.75%,90.25%


In [29]:
# связаны ли цели кредита с семейным статусом?
df_family_purpose = df.pivot_table(index=['family_status',], columns='purpose_general', values='dob_years', aggfunc='count') # создание сводной таблицы, где каждому семейному положению соответствует количесвто кредитов по каждой цели 
df_family_purpose = df_family_purpose.fillna(value='-') # замена пропущенных значений на "-"
print(df_family_purpose)

purpose_general        автомобиль  недвижимость  образование свадьба
family_status                                                       
в разводе                   281.0         676.0        238.0       -
вдовец / вдова              218.0         542.0        199.0       -
гражданский брак            434.0         989.0        404.0    2324
женат / замужем            2736.0        7008.0       2595.0       -
не женат / не замужем       637.0        1596.0        577.0       -


**Вывод**

Факт того, состоит или нет человек в отношениях, не сильно влияет на вероятность того, что он не отдаст кредит (как люди в гражданском браке, так и люди, которые не женаты/не замужен, в примерно 9% случаях являются должниками. но при этом люди в разводе и женатые/замужние люди являются долдниками примерно в 7% случаев). Меньше всего кредитов берут вдовы или вдовцы, скорее всего из-за меньшей финансовой поддержки, но при этом лучше всего отдают кредиты. Больше всего кредитов берут люди в браке, так как в браке появляется больше материальных потребностей (покупка машины, ипотека), но часто являются должниками.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
df_income_debt = df.pivot_table(index=['income_type'], columns='debt', values='dob_years', aggfunc='count') # 
df_income_debt[0] = df_income_debt[0].fillna(value=0) # заполнение NaN в столбце 0 нулями
df_income_debt[1] = df_income_debt[1].fillna(value=0) # заполнение NaN в столбце 1 нулями
df_income_debt['debt_percent'] = df_income_debt.apply(percent_debt, axis=1)

In [39]:
# сотрудники и госслужащие получают примерно одинаковую зарплату, но среди сотрудников почти в 2 раза больше должников. Почему?
# связан ли долг по кредиту с целями кредита по разным категориям дохода? Берут ли сотрудники более дорогие кредиты по сравнению с госслужащими?
df_income_purpose = df.pivot_table(index=['income_type'], columns='purpose_general', values='dob_years', aggfunc='count') # сводная таблица по категориям дохода и целям кредита
df_income_purpose = df_income_purpose.fillna(value='-') # замена пустых значений на прочерк
df_income_purpose['median_income'] = df.groupby('income_type')['total_income'].median()
print(df_income_purpose)
# распределение целей кредитов примерно одинаковое во всем категориям примерно одинаковое 

purpose_general автомобиль недвижимость образование свадьба  median_income
income_type                                                               
безработный              -            2           -       -         131339
в декрете                1            -           -       -          53829
госслужащий            286          754         258     159         150447
компаньон             1052         2547         954     525         172357
пенсионер              795         1891         721     422         118514
предприниматель          -            1           -       1         499163
сотрудник             2172         5615        2080    1217         142594
студент                  -            1           -       -          98201


**Вывод**

Количество предпринимателей, студентов, людей в декрете и безработных слишком мало, поэтому при анализе зависимости между уровнем дохода и возвратом кредита в срок эти строки в таблице df_income_debt можно не учитывать. Все оставшиеся категории находятся в средней категории по уровню дохода. Среди этих групп по распределение по целям кредита примерно одинаковое (то есть группы с высоким процентом должников не берут более дорогие кредиты по сравнению с группами, с низким процентом должников). Ниже всего процент должников среди пенсионеров и госслужащихБ больше всего - среди сотрудников. Несмотря на то, что компаньоны и сотрудники получают примерно одинаковую зарплату, сотрудники почти в два раза чаще являются должниками. Возможно, банк предлагает сотрудникам плохие кредитные условия. Госслужащим и пенсионерам банк может выдавать кредит по льготным условия, в результате чего в этих категориях низкий процент должников.

- Как разные цели кредита влияют на его возврат в срок?

In [31]:
df_purpose_debt = df.pivot_table(index=['purpose_general'], columns='debt', values='dob_years', aggfunc='count')
df_purpose_debt['debt_percent'] = df_purpose_debt.apply(percent_debt, axis=1)
print(df_purpose_debt)

debt                 0    1 debt_percent
purpose_general                         
автомобиль        3903  403        9.36%
недвижимость     10029  782        7.23%
образование       3643  370        9.22%
свадьба           2138  186         8.0%


**Вывод**

Больше всего процентов должников по автокредитам и по кредитам на образование. Возможно, у банка очень невыгодные условия для автокредитов (например, большие проценты) из-за чего заемщику тяжело отдавать такой кредит, поэтому имеет смысл пересмотреть условия автокредита. Высокий процент должников по кредиту за образование может быть связан с тем, что банки выдают кредит на образование без справки о доходах или с тем, что заемщики не могут найти работу после обучения, соответственно, им нечем выплачивать кредит. Необходимо требовать документы о доходах от людей, желающих взять кредит на образрвание.

## Шаг 4. Общий вывод

Надежнее всего давать кредиты бездетным людям, так как с увеличением количества детей увеличивается шанс того, что заемщик не отдаст кредит.
Между семейным положением и возравратом кредита в срок нет зависимости.
Недостаточно информации по людям с дохом ниже 100 тысяч и выше 200 тысяч. Среди сотрудников почти в два раза больше должников по сравнению с другими категориями заработка. Возможно стоит пересмотреть условия кредитования для людей из этой категории.
В процентном соотношении меньше всего задолжденностей по кредитам на жилье. Вероятно, это связано с тем, что можно получить кредит на жилье на льготных условиях. Так как больше всего должников по автокредитам и кредитам на образование, то стоит пересмотреть условия этих кредитов и просить у клиентов, собирающихся брать эти кредиты, справки о доходах.